In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.ex.exglobal as exglo
from freqopttest.ex.ex4_text import load_nips_TSTData
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import freqopttest.glo as glo
import freqopttest.plot as plot
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import sys

In [ ]:
#fname = 'ex5-S48_HANESU_AFANDI-me3_J2_rs300_nma419_d1680_a0.010_trp0.50.p'

#fname = 'ex5-crop48_HANESU_AFANDI-me6_J1_rs500_nma402_d1632_a0.010_trp0.50.p'
#fname = 'ex5-crop48_h0-me6_J1_rs500_nma402_d1632_a0.010_trp0.50.p'
#fname = 'ex5-crop48_h0-me6_J1_rs200_nma402_d1632_a0.010_trp0.50.p'
fname = 'ex5-crop48_HANESU_AFANDI-me6_J1_rs200_nma402_d1632_a0.010_trp0.50.p'

ex = 5
results = glo.ex_load_result(ex, fname)

In [ ]:
data_fname = results['data_fname']
#labels = ['ME-full', 'ME-opt-0.5', 'ME-full', 'ME-gw-opt', 
#        'ME-grid', 'SCF-full', 'SCF-full', 'SCF-gw-opt', 'SCF-grid',
#        'MMD-lin', '$T^2$']

method = 'ME-full'
method_job_funcs = results['method_job_funcs']
func_names = [f.__name__ for f in method_job_funcs]
func2labels = exglo.get_func2label_map()
method_labels = [func2labels[f] for f in func_names if f in func2labels]

method_index = method_labels.index(method)

results0 = results['results'] 
method_results = results0[:, method_index]

alpha = 0.01
reps = len(method_results)

In [ ]:
def methods_powers(R, reps):
    """Return the powers of all methods"""
    #n_methods = len(R['method_labels'])
    n_methods = len(R['method_job_funcs'])
    met_powers = np.zeros(n_methods)
    results0 = R['results'] 
    for mi in range(n_methods):
        method_results = results0[:, mi]
        pvals = np.array([method_results[r]['test_result']['pvalue'] for r in range(reps)] )
        met_powers[mi] = np.mean(pvals < alpha)
    return met_powers

def methods_runtimes(R, reps):
    """Return the runtimes of all methods"""
    n_methods = len(R['method_job_funcs'])
    times = np.zeros(n_methods)
    results0 = R['results'] 
    for mi in range(n_methods):
        method_results = results0[:, mi]
        ts = np.array([method_results[r]['time_secs'] for r in range(reps)] )
        times[mi] = np.mean(ts)
    return times

In [ ]:
met_pows = methods_powers(results, reps)
met_times = methods_runtimes(results, reps)
print('fname: %s'%fname)
print(method_labels)
print('test powers: ')
print(met_pows)
print('runtimes: ')
print(met_times)


In [ ]:
img_height = 48

# Get all learned test locations
def collect_test_locations(method_results, loc_ind):
    return np.array([method_results[r]['test_method'].test_locs[loc_ind] for r in range(reps) ])

def show_loc_img(loc, vmin=0, vmax=1):
    loc_img = np.reshape(loc, (img_height, -1))
    # cmap reference: http://matplotlib.org/examples/color/colormaps_reference.html
    return plt.imshow(loc_img, 
                      #cmap='seismic',
                      #cmap='gray',
                      #cmap='Greys',
                      cmap='afmhot',
                     vmin=vmin, vmax=vmax, 
                      #interpolation='mitchell')
                      interpolation='None')
    
locs0 = collect_test_locations(method_results, 0)
mean_loc0 = np.mean(np.abs(locs0), 0)


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
#show_loc_img(mean_loc0, vmin=np.min(mean_loc0), vmax=np.max(mean_loc0))
show_loc_img( np.abs(mean_loc0), vmin=0.25, vmax=np.max(mean_loc0))
#plt.colorbar()
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.gca().set_frame_on(False)

plt.savefig('learned_face.pdf', bbox_inches='tight')

In [ ]:
"""
locs1 = collect_test_locations(method_results, 1)
mean_loc1 = np.mean(locs1, 0)

plt.subplot(1, 2, 2)
show_loc_img(mean_loc1, vmin=0, vmax=0.6)
plt.colorbar()
"""

In [ ]:
plt.figure(figsize=(12, 10))
n_imgs = 15
n_cols = 5
for i in range(n_imgs): 
    plt.subplot(n_imgs/n_cols, n_cols, i+1)
    imgplot = show_loc_img(np.abs(locs0[i]), vmin=0, vmax=1)
plt.colorbar()


## Data analysis

In [ ]:
loaded = glo.load_data_file(results['data_fname'])
if isinstance(loaded, np.ndarray):
    ss = data.SSNullResample(loaded)
    tst_data = ss.sample(loaded.shape[0]/2, seed=29)
    X, Y = tst_data.xy()
elif isinstance(loaded, tst.TSTData):
    X, Y  = loaded.xy()
    

In [ ]:
# plot samples of the data
inds = np.array([0, 1, 3])
plt.figure(figsize=(12, 10))
n_imgs = 6
n_cols = 3
for i in range(n_imgs): 
    plt.subplot(n_imgs/n_cols, n_cols, i+1)
    show_loc_img(X[i], vmin=0, vmax=np.max(X[i]))
    plt.gca().get_xaxis().set_visible(False)
    plt.gca().get_yaxis().set_visible(False)
    plt.gca().set_frame_on(False)



In [ ]:
mean_x = np.mean(X, 0)
mean_y = np.mean(Y, 0)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
show_loc_img(mean_x, vmax=0.5)
plt.title('positive emotion')

plt.subplot(1, 2, 2)
show_loc_img(mean_y, vmax=0.5)
plt.title('negative emotion')

In [ ]:
# classify each location according to the distance to X (0) or Y (1)
centroids = np.vstack((mean_x, mean_y))
#def nearest_centroid(centroids, locs):
D = util.dist_matrix(locs0, centroids)



In [ ]:
show_loc_img(mean_x - mean_y, vmin=0, vmax=0.1)